In [1]:
%load_ext autoreload
%autoreload 1

In [57]:
%aimport common_google_drive
# from googleapiclient.discovery import build
# from googleapiclient.http import MediaIoBaseDownload
# from google_auth_oauthlib.flow import InstalledAppFlow
# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# import io
import pandas as pd
from pymongo import MongoClient
import os
from datetime import datetime
import re
import tqdm

creds = common_google_drive.get_creds(
    client_secret_file="2021-04-23-music-lib/client_secret.json",
    create_if_not_exist=True,
#     token
)

In [70]:
tasks_df = common_google_drive.download_df_from_google_sheets(
    creds,
    spreadsheet_id="1xFndlfiywpGO1H7DYSP3LEdljESNtj4SwLHBUrmj1i4",
    sheet_name="pool of weekend tasks"
)

tasks_df.scheduled_date = tasks_df.scheduled_date.apply(lambda s:None if len(s)==0 else datetime.strptime(s,"%Y/%m/%d"))
tasks_df.head()

def _due_processor(s):
    if len(s)==0:
        return None
    m = re.match(r"(?P<year>\d{4})/(?P<month>\d{2})/(?P<day>\d{2}) (?P<hour>\d{1,2}):(?P<minute>\d{1,2})(:(?P<second>\d{1,2}))?",s)
    assert m is not None,s
    kwargs = {k:int(m.group(k)) for k in "year,month,day,hour,minute".split(",")}
    kwargs["second"] = 0 if m.group("second") is None else int(m.group("second"))
    return datetime(*list(kwargs.values()))

tasks_df.due = tasks_df.due.apply(_due_processor)
tasks_df.status = tasks_df.status.apply(lambda s:s if s in "DONE,REGULAR,FAILED".split(",") else None)
tasks_df.URL = tasks_df.URL.apply(lambda s:None if len(s)==0 else s)
tasks_df.when = tasks_df.when.apply(lambda s:s if s in "WEEKEND,EVENING,PARTTIME".split(",") else None)
tasks_df.head()

,name,URL,scheduled_date,status,when,due,uuid
0,try лечебное голодание,None,1991-12-24,DONE,None,NaT,476ef677-07e3-4851-9c4d-b8de0a191ee4
1,regular weekly srm test,None,1991-12-24,REGULAR,WEEKEND,NaT,f73bd7bc-9415-4cb3-ba07-5c80a384792b
2,"switch golod to monday, not friday",None,1991-12-24,DONE,WEEKEND,NaT,6db4f09b-2259-4678-a0c1-013f8e8f8c66
3,do one exercise laps per eve ring routine,None,1991-12-24,REGULAR,EVENING,NaT,d0ccbf4d-a2bd-49f9-977c-20bb3b01722c
4,start coursera,None,1991-12-24,REGULAR,EVENING,NaT,a0ee2bc4-4f82-42ff-a520-a35682ea9a3f


In [71]:
assert len(tasks_df[[len(u)!=len("476ef677-07e3-4851-9c4d-b8de0a191ee4") or u is None for u in tasks_df.uuid]])==0
tasks_df.status.unique()

array(['DONE', 'REGULAR', 'FAILED', None], dtype=object)

In [73]:
client = MongoClient(os.environ["PYASSISTANTBOT_MONGO_URL"])
for r in tqdm.notebook.tqdm(tasks_df.to_dict(orient="records")):
    for k in ["due","scheduled_date"]:
        if pd.isna(r[k]):
            r[k] = None
    client.gstasks.tasks.insert_one(r)